In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import argparse
import re
import time
from datetime import timedelta
from functools import partial
import logging
from datasets import load_dataset, Dataset
from tqdm import tqdm
import os
from torch.utils.data import DataLoader

from src.retriever import Retriever
from types import SimpleNamespace

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [3]:

arg = SimpleNamespace()
arg.retrieval_model_name_or_path="facebook/contriever-msmarco"
arg.retrieval_embedding_size=768
arg.passages='data/corpus/psgs_w100.tsv'
arg.passages_embeddings='data/corpus/wikipedia_embeddings/*'
arg.indexing_batch_size=1000000
arg.save_or_load_index = False
arg.retrieval_n_subquantizers=0
arg.retrieval_n_bits=8
arg.max_k=100
arg.lowercase = False
arg.normalize_text = False
arg.per_gpu_batch_size=1000000
arg.question_maxlength=100000

retriever = Retriever(arg)
retriever.setup_retriever()

Loading model from: facebook/contriever-msmarco


/home/tkdrnjs0621/miniconda3/envs/torch_241/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Indexing passages from files ['data/corpus/wikipedia_embeddings/passages_00', 'data/corpus/wikipedia_embeddings/passages_01', 'data/corpus/wikipedia_embeddings/passages_02', 'data/corpus/wikipedia_embeddings/passages_03', 'data/corpus/wikipedia_embeddings/passages_04', 'data/corpus/wikipedia_embeddings/passages_05', 'data/corpus/wikipedia_embeddings/passages_06', 'data/corpus/wikipedia_embeddings/passages_07', 'data/corpus/wikipedia_embeddings/passages_08', 'data/corpus/wikipedia_embeddings/passages_09', 'data/corpus/wikipedia_embeddings/passages_10', 'data/corpus/wikipedia_embeddings/passages_11', 'data/corpus/wikipedia_embeddings/passages_12', 'data/corpus/wikipedia_embeddings/passages_13', 'data/corpus/wikipedia_embeddings/passages_14', 'data/corpus/wikipedia_embeddings/passages_15']
Loading file data/corpus/wikipedia_embeddings/passages_00
Total data indexed 1000000
Loading file data/corpus/wikipedia_embeddings/passages_01
Total data indexed 2000000
Loading file data/corpus/wikiped

In [10]:
retriever.reset_args(arg)

In [4]:
from datasets import Dataset
dataset = Dataset.from_json("data/eval_data/triviaqa_test.jsonl")

In [11]:
idx = 0
print(dataset[idx]['question'])
print(dataset[idx]['answers'])
rs = retriever.search_document(dataset[idx]['question'], top_n=10)
for a in rs:
    print(a)

Who was the man behind The Chipmunks?
['David Seville', 'david seville']


AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'cpu'